In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#Read the file
import pandas as pd
df = pd.read_csv('/kaggle/input/startup-logistic-regression/50_Startups.csv')
print(df.head())
df.to_pickle('mlr_dataset.pkl')
# Print the different values in the State column, and the no. of rows having that value
print(df["State"].value_counts())

In [2]:
#Visualizing the dataset
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette('colorblind')
sns.pairplot(data=df, height=3, hue='State')

In [3]:
#Split the dataset into input features and target labels
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [5]:
# Convert text data to numbers
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [7]:
#Scale the dataset
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)
print(x)

In [8]:
#Dividing into train and test set
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split( x, y, test_size = 0.3,random_state=42)

In [11]:
#Defining cost and gradient function
def CostFunction(x,y,w,b):
    cost = np.sum((((x.dot(w) + b) - y) ** 2) / (2*len(y)))
    return cost

def GradientDescent(x, y, w, b, learning_rate, epochs):
    cost_list = [0] * epochs
   
    for epoch in range(epochs):
        z = x.dot(w) + b
        loss = z - y
        
        weight_gradient = x.T.dot(loss) / len(y)
        bias_gradient = np.sum(loss) / len(y)
        
        w = w - learning_rate*weight_gradient
        b = b - learning_rate*bias_gradient
  
        cost = CostFunction(x, y, w, b)
        cost_list[epoch] = cost
        
        if (epoch%(epochs/10)==0):
            print("Cost for epoch -",epoch, "is:", cost)
        
    return w, b, cost_list

In [12]:
#Calculating the gradients
w, b, c= GradientDescent(Xtrain, Ytrain, np.zeros(Xtrain.shape[1]), 0, 0.002,epochs=15000)

In [13]:
#Plotting the cost
import matplotlib.pyplot as plt
plt.plot(c)

In [14]:
#Predicting labels with calculated weights and biases
def predict(X, w, b):
    return X.dot(w) + b
y_pred = predict(Xtest, w, b)

In [15]:
#Calculating r2 score, to check accuracy of prediction
def r2score(y_pred, y):
    rss = np.sum((y_pred - y) ** 2)
    tss = np.sum((y-y.mean()) ** 2)
    
    r2 = 1 - (rss / tss)
    return r2
r2score(y_pred, Ytest)